In [11]:
import subprocess
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pickle

# Chrome 브라우저를 원격 디버깅 모드로 실행
chrome_path = "C:/Program Files/Google/Chrome/Application/chrome.exe"
chrome_debug_port = "9222"
user_data_dir = "C:/Selenium/ChromeData"

# Chrome을 원격 디버깅 모드로 실행
subprocess.Popen([chrome_path, f'--remote-debugging-port={chrome_debug_port}', f'--user-data-dir={user_data_dir}'])

# Selenium 옵션 설정
options = Options()
options.add_experimental_option("debuggerAddress", f"127.0.0.1:{chrome_debug_port}")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 구글 로그인 페이지로 이동
driver.get('https://accounts.google.com/signin')

# 수동으로 로그인 진행
print("Please log in manually and then press Enter here...")
input("Press Enter after you have logged in...")  # 터미널에서 엔터를 누름

# 로그인된 쿠키 저장
cookies = driver.get_cookies()
with open('cookies.pkl', 'wb') as file:
    pickle.dump(cookies, file)

print("Cookies saved.")
driver.quit()


In [6]:
import subprocess
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pickle
from selenium.webdriver.common.keys import Keys

# Chrome 브라우저를 원격 디버깅 모드로 실행
chrome_path = "C:/Program Files/Google/Chrome/Application/chrome.exe"
chrome_debug_port = "9222"
user_data_dir = "C:/Selenium/ChromeData"

# Chrome을 원격 디버깅 모드로 실행
subprocess.Popen([chrome_path, f'--remote-debugging-port={chrome_debug_port}', f'--user-data-dir={user_data_dir}'])

# Selenium 옵션 설정
options = Options()
options.add_experimental_option("debuggerAddress", f"127.0.0.1:{chrome_debug_port}")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # 페이지 접속
    driver.get('https://livewiki.com/ko')
    time.sleep(1)
    driver.maximize_window()
    time.sleep(3)
    # 쿠키 로드
    with open('cookies.pkl', 'rb') as file:
        cookies = pickle.load(file)

    # 쿠키를 현재 페이지에 추가
    for cookie in cookies:
        if 'domain' in cookie:
            cookie['domain'] = 'livewiki.com'
        driver.add_cookie(cookie)

    # 페이지 새로 고침하여 로그인 상태 확인
    driver.refresh()

    # 데이터 저장 리스트 및 실패한 URL 저장 리스트
    all_data = []
    failed_urls = []

    # URL 리스트 파일 경로
    path_list_file = '2024.csv'

    # URL 리스트 파일 읽기 (헤더가 없는 경우)
    path_list = pd.read_csv(path_list_file, header=None)[0].tolist()
    
    # 처음 500개의 URL을 선택
    path_list = path_list[:500]

    for url in path_list:
        try:
            # URL 입력 필드에 값 입력
            url_input = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#__next > div > div.css-u47me4.e1pa7md80 > div > header > header > div.css-1l2dl5u.e15utfnb3 > div > div > input[type=text]"))
            )
            driver.execute_script("arguments[0].value = '';", url_input)
            time.sleep(3)
            url_input.send_keys(url)
            time.sleep(3)

            url_input.send_keys(Keys.RETURN)
            
            # 페이지 로딩 대기
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "span[style='width: fit-content; height: fit-content;']"))
            )
            time.sleep(10)
            # 데이터 추출
            video_titles = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'div.css-1c1xn96.es80ps51')]
            video_contents = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'div.markdown-body')]

            # 데이터 길이 맞추기
            max_len = max(len(video_titles), len(video_contents))
            video_titles.extend([''] * (max_len - len(video_titles)))
            video_contents.extend([''] * (max_len - len(video_contents)))

            # 데이터프레임 생성
            data = {
                'title': video_titles,
                'contents': video_contents
            }

            df = pd.DataFrame(data)
            all_data.append(df)

            # 입력 필드 클리어
            driver.execute_script("arguments[0].value = '';", url_input)

        except Exception as e:
            print(f"Error occurred for URL {url}: {e}")
            failed_urls.append(url)  # 실패한 URL을 저장

        time.sleep(3)

finally:
    # 드라이버 종료
    driver.quit()

    # DataFrame으로 합치기
    if all_data:
        all_data_df = pd.concat(all_data, ignore_index=True)
        all_data_df.to_csv('2024_result.csv', index=False, encoding='utf-8')

    # 실패한 URL을 CSV 파일로 저장
    if failed_urls:
        pd.DataFrame(failed_urls, columns=['failed_urls']).to_csv('failed_urls.csv', index=False, encoding='utf-8')

    print("크롤링이 완료되었습니다.")
    if failed_urls:
        print(f"총 {len(failed_urls)}개의 URL에서 데이터 추출에 실패했습니다.")


크롤링이 완료되었습니다.


AttributeError: 'str' object has no attribute 'head'